# lab04-2 Loading Data from File

### 2018.09.22(토)
__NumPy를 이용하여 Data를 Load하고, Handling 한다.__

import tensorflow as tf
import numpy as np

xy = np.loadtxt('data-01-test-score.csv',delimiter = ",", dtype = np.float32)# dtpe을 정해주는 것의 단점? => csv내의 모든 데이터 타입이 동일해야 한다.
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

# Make sure the shape and data OK
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

In [39]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape = [None, 3])
Y = tf.placeholder(tf.float32, shape = [None, 1])

W = tf.Variable(tf.random_normal([3,1]),name = "weight")
b = tf.Variable(tf.random_normal([1]),name = "bias")

# hypothesis
hypothesis = tf.matmul(X, W) + b

# simplified cost/Loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 1e-5)
train = optimizer.minimize(cost)

In [40]:
# Launch the graph in a session
sess = tf.Session()
# Initializes global variables in the graph
sess.run(tf.global_variables_initializer())
# Set up feed_dict variables inside the loop
for step in range(2001):
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train],
        feed_dict = {X: x_data, Y: y_data})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val,
                "\nPrediction: \n", hy_val)

# Ask my score
print("Your score will be ",sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))
print("Other scores will be ",sess.run(hypothesis, feed_dict={X: [[60,70, 110],[90,100,80]]}))

0 Cost:  64456.887 
Prediction: 
 [[ -75.46573 ]
 [-108.29259 ]
 [ -97.66551 ]
 [-106.95099 ]
 [ -86.312485]
 [ -66.26047 ]]
10 Cost:  58.85519 
Prediction: 
 [[161.08536]
 [176.1655 ]
 [182.54149]
 [198.2061 ]
 [130.67723]
 [ 93.71935]]
20 Cost:  55.558743 
Prediction: 
 [[162.65292]
 [178.10916]
 [184.42552]
 [200.26248]
 [132.17067]
 [ 94.83443]]
30 Cost:  55.218082 
Prediction: 
 [[162.62918]
 [178.13945]
 [184.42465]
 [200.26617]
 [132.20459]
 [ 94.8735 ]]
40 Cost:  54.87981 
Prediction: 
 [[162.59477]
 [178.15672]
 [184.41106]
 [200.256  ]
 [132.22855]
 [ 94.90517]]
50 Cost:  54.543903 
Prediction: 
 [[162.56041 ]
 [178.17383 ]
 [184.39745 ]
 [200.24576 ]
 [132.25235 ]
 [ 94.936676]]
60 Cost:  54.21007 
Prediction: 
 [[162.52615]
 [178.19089]
 [184.38385]
 [200.23553]
 [132.2761 ]
 [ 94.96806]]
70 Cost:  53.878407 
Prediction: 
 [[162.49199]
 [178.20789]
 [184.3703 ]
 [200.22534]
 [132.29974]
 [ 94.99935]]
80 Cost:  53.549023 
Prediction: 
 [[162.45798]
 [178.22484]
 [184.35683]


## Queue Runners

__불러들이고자 하는 file의 메모리가 너무 큰 경우, numpy를 써서 올리기 힘들다__
--> 이를 해결하기 위해, TensorFlow 에서는 Queue Runner라는 시스템을 만듦.
![Queue](QueueRunners.gif)

1. 여러개의 Filename
2. Filename Queue에 filename을 저장
3. 이를 Reader로 연결하여 양식에 맞게 Decoder로 처림
4. 각 batch만큼 data를 읽어와서 Example Queue 에서 액션을 처리.

----

Step1. 파일 명을 정리하여 이로 구성된 list를 tf.train.string_input_producer
<br>
Step2. Textline을 읽을 reader를 준비해 놓는다. reader = tf.TextLineReader()
<br>
Step3. 읽어온 Data에 대한 parsing (How..?)

In [41]:
tf.train.batch

<function tensorflow.python.training.input.batch>

In [56]:
# collect batchs of csv in
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]],batch_size = 10) #batch는 일종의 펌프같이 데이터를 데려와서 처리하게 함
sess = tf.Session()


In [57]:
# Start populating the filename queue
# 상기 이미지와 관련하여 복잡한 부분은 tensorflow가 알아서 다 해준다.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord)


In [58]:
for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    # 차후에 x_batch 와 y_batch는 feed_dict를 통해서 값을 넘겨주면 된다.

In [59]:
coord.request_stop()
coord.join(threads)

### 이를 이용한 전체 코드

In [71]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(
        ["data-01-test-score.csv"],shuffle = False, name = "filename_queue")

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also spectified the type of the decoded result.
record_defaults = [[0.],[0.],[0.],[0.]]
xy = tf.decode_csv(value, record_defaults = record_defaults)
# record_defaults의 경우, 각각의 datatype을 정해주는데, 이때 각각 [0.]. 즉, 실수임을 명명한다.

# collect batches of csv in
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1],xy[-1:]], batch_size = 10)
# 데이터를 한번에 10개씩 가져와서 train_batch에 저장

In [68]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None,3])
Y = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([3,1]),name = "weight")
b = tf.Variable(tf.random_normal([1]), name = "bias")
# Shape 에 주의

# hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/Loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [72]:
# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph
sess.run(tf.global_variables_initializer())

# Start populating the filename queue
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess = sess, coord = coord)
# 이부분은 일반적으로 쓰는 부분( 열고 닫는다고 생각 )

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train],
        feed_dict = {X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost : ", cost_val,
             "\nPrediction : \n", hy_val)
        
coord.request_stop()
coord.join(threads)

0 Cost :  5737.0864 
Prediction : 
 [[224.6332 ]
 [264.29495]
 [263.8221 ]
 [281.3737 ]
 [205.0929 ]
 [141.42256]
 [224.6332 ]
 [264.29495]
 [263.8221 ]
 [281.3737 ]]
10 Cost :  13.35161 
Prediction : 
 [[143.1477 ]
 [ 95.76603]
 [157.05794]
 [183.0801 ]
 [183.79683]
 [194.2282 ]
 [143.1477 ]
 [ 95.76603]
 [157.05794]
 [183.0801 ]]
20 Cost :  12.119688 
Prediction : 
 [[183.35861]
 [193.75452]
 [142.82048]
 [ 95.53296]
 [156.67911]
 [182.64746]
 [183.35861]
 [193.75452]
 [142.82048]
 [ 95.53296]]
30 Cost :  11.7982855 
Prediction : 
 [[156.72862 ]
 [182.7292  ]
 [183.4274  ]
 [193.83298 ]
 [142.88545 ]
 [ 95.588905]
 [156.72862 ]
 [182.7292  ]
 [183.4274  ]
 [193.83298 ]]
40 Cost :  13.1282 
Prediction : 
 [[142.77632 ]
 [ 95.516426]
 [156.58821 ]
 [182.58266 ]
 [183.27129 ]
 [193.6664  ]
 [142.77632 ]
 [ 95.516426]
 [156.58821 ]
 [182.58266 ]]
50 Cost :  11.978087 
Prediction : 
 [[183.33775]
 [193.74232]
 [142.83978]
 [ 95.57131]
 [156.63559]
 [182.66226]
 [183.33775]
 [193.74232]
 [

### Shuffle_batch
__ batch의 순서를 shuffle 해서 적용 __

In [74]:
# min_after_dequeue defines how big a buffer we will randomly sample
#  from -- bigger means better shuffling but slower start up and more memory used.
# capacity must be Larger than min_after_dequeue and the amount Larger
#  determinies the maximum we will prefetch. Recommendation:
#  min_after_dequeue + (num_threads + a small safety margin) * batch_size

min_after_dequeue = 10000
capacity = min_after_dequeue + 3 * batch_size
example_batch, label_batch = tf.train.shuffle_batch(
    [example, label], batch_size = batch_size, capacity = capacity, min_after_dequque = min_after_dequque
)

NameError: name 'batch_size' is not defined